In [1]:
from dataset import VikramDataset
import transforms as t

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [12]:
NUM_EPOCHS = 30
BATCH_SIZE = 64
lr = 0.001

In [3]:
# preprocessing
train_transform = t.Compose([
    t.AddFlanks("",""),
    t.LeftCrop(230,230),
    t.Seq2Tensor(),
    t.Reverse(),
    t.AddReverseChannel()

])
test_transform = t.Compose([
    t.AddFlanks("",""),
    t.LeftCrop(230,230),
    t.Seq2Tensor(),
    t.AddReverseChannel()
])

# load the data
train_dataset = VikramDataset(cell_type = "HepG2", split="train", transform=train_transform) # could use a list e.g. [1,2,5,6,7,8] 
                                                                                            # for needed folds
val_dataset = VikramDataset(cell_type = "HepG2", split="val", transform=test_transform) # use "val" for default validation set
test_dataset = VikramDataset(cell_type = "HepG2", split="test", transform=test_transform) # use "test" for default test set

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [4]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

def initialize_weights(m):
    if isinstance(m, nn.Conv1d):
        n = m.kernel_size[0] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2 / n))
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight.data, 1)
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        m.weight.data.normal_(0, 0.001)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)

class SELayer(nn.Module):
    def __init__(self, inp, reduction=4):
        super(SELayer, self).__init__()
        self.fc = nn.Sequential(
                nn.Linear(inp, int(inp // reduction)),
                nn.SiLU(),
                nn.Linear(int(inp // reduction), inp),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, = x.size()
        y = x.view(b, c, -1).mean(dim=2)
        y = self.fc(y).view(b, c, 1)
        return x * y

class EffBlock(nn.Module):
    def __init__(self, in_ch, ks, resize_factor, activation, out_ch=None, se_reduction=None):
        super().__init__()
        self.in_ch = in_ch
        self.out_ch = self.in_ch if out_ch is None else out_ch
        self.resize_factor = resize_factor
        self.se_reduction = resize_factor if se_reduction is None else se_reduction
        self.ks = ks
        self.inner_dim = self.in_ch * self.resize_factor

        block = nn.Sequential(
                        nn.Conv1d(
                            in_channels=self.in_ch,
                            out_channels=self.inner_dim,
                            kernel_size=1,
                            padding='same',
                            bias=False
                       ),
                       nn.BatchNorm1d(self.inner_dim),
                       activation(),

                       nn.Conv1d(
                            in_channels=self.inner_dim,
                            out_channels=self.inner_dim,
                            kernel_size=ks,
                            groups=self.inner_dim,
                            padding='same',
                            bias=False
                       ),
                       nn.BatchNorm1d(self.inner_dim),
                       activation(),
                       SELayer(self.inner_dim, reduction=self.se_reduction),
                       nn.Conv1d(
                            in_channels=self.inner_dim,
                            out_channels=self.in_ch,
                            kernel_size=1,
                            padding='same',
                            bias=False
                       ),
                       nn.BatchNorm1d(self.in_ch),
                       activation(),
        )

        self.block = block

    def forward(self, x):
        return self.block(x)

class LocalBlock(nn.Module):
    def __init__(self, in_ch, ks, activation, out_ch=None):
        super().__init__()
        self.in_ch = in_ch
        self.out_ch = self.in_ch if out_ch is None else out_ch
        self.ks = ks

        self.block = nn.Sequential(
                       nn.Conv1d(
                            in_channels=self.in_ch,
                            out_channels=self.out_ch,
                            kernel_size=self.ks,
                            padding='same',
                            bias=False
                       ),
                       nn.BatchNorm1d(self.out_ch),
                       activation()
        )

    def forward(self, x):
        return self.block(x)

class ResidualConcat(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        return torch.concat([self.fn(x, **kwargs), x], dim=1)

class MapperBlock(nn.Module):
    def __init__(self, in_features, out_features, activation=nn.SiLU):
        super().__init__()
        self.block = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Conv1d(in_channels=in_features,
                      out_channels=out_features,
                      kernel_size=1),
        )

    def forward(self, x):
        return self.block(x)

class HumanLegNet(nn.Module):
    def __init__(self,
                 in_ch,
                 stem_ch,
                 stem_ks,
                 ef_ks,
                 ef_block_sizes,
                 pool_sizes,
                 resize_factor,
                 activation=nn.SiLU,
                 ):
        super().__init__()
        assert len(pool_sizes) == len(ef_block_sizes)

        self.in_ch = in_ch
        self.stem = LocalBlock(in_ch=in_ch,
                               out_ch=stem_ch,
                               ks=stem_ks,
                               activation=activation)

        blocks = []

        in_ch = stem_ch
        out_ch = stem_ch
        for pool_sz, out_ch in zip(pool_sizes, ef_block_sizes):
            blc = nn.Sequential(
                ResidualConcat(
                    EffBlock(
                        in_ch=in_ch,
                        out_ch=in_ch,
                        ks=ef_ks,
                        resize_factor=resize_factor,
                        activation=activation)
                ),
                LocalBlock(in_ch=in_ch * 2,
                           out_ch=out_ch,
                           ks=ef_ks,
                           activation=activation),
                nn.MaxPool1d(pool_sz) if pool_sz != 1 else nn.Identity()
            )
            in_ch = out_ch
            blocks.append(blc)
        self.main = nn.Sequential(*blocks)

        self.mapper = MapperBlock(in_features=out_ch,
                                  out_features=out_ch * 2)
        self.head = nn.Sequential(nn.Linear(out_ch * 2, out_ch * 2),
                                   nn.BatchNorm1d(out_ch * 2),
                                   activation(),
                                   nn.Linear(out_ch * 2, 1))

    def forward(self, x):
        x = self.stem(x)
        x = self.main(x)
        x = self.mapper(x)
        x =  F.adaptive_avg_pool1d(x, 1)
        x = x.squeeze(-1)
        x = self.head(x)
        x = x.squeeze(-1)
        return x

In [14]:
import pytorch_lightning as L
from torchmetrics import MeanSquaredError
from torchmetrics import PearsonCorrCoef

class SeqModel(L.LightningModule):
    
    def __init__(self, in_ch, lr=3e-4, onecycle=False):
        super().__init__()
        self.model = HumanLegNet(in_ch=in_ch,
                                 stem_ch=64,
                                 stem_ks=11,
                                 ef_ks=9,
                                 ef_block_sizes=[80, 96, 112, 128],
                                 pool_sizes=[2,2,2,2],
                                 resize_factor=4)
        self.criterion = nn.MSELoss() 
        self.lr = lr
        self.val_mse = MeanSquaredError()
        self.val_pearson = PearsonCorrCoef()
        self.onecycle = onecycle
        
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_nb):
        x, y = batch
        y = y.float()
        loss = self.criterion(self(x), y)
        self.log("train_loss", loss, prog_bar=True)
        return loss
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.float()
        preds = self(x)
        self.val_mse.update(preds, y)

        self.log("val_acc", self.val_mse, prog_bar=True)
        self.val_pearson(preds, y)
        self.log("val_pearson", self.val_pearson, on_epoch=True, prog_bar = True)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        if isinstance(batch, tuple) or isinstance(batch, list):
            x, _ = batch
        else:
            x = batch
        return self(x)

    def configure_optimizers(self):
        if self.onecycle:
            optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr / 50)
            lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                            total_steps=self.trainer.estimated_stepping_batches,
                                                            max_lr=0.01)
            lr_scheduler_config = {
                    # REQUIRED: The scheduler instance
                    "scheduler": lr_scheduler,
                    # The unit of the scheduler's step size, could also be 'step'.
                    # 'epoch' updates the scheduler on epoch end whereas 'step'
                    # updates it after a optimizer update.
                    "interval": "step",
                    # How many epochs/steps should pass between calls to
                    # `scheduler.step()`. 1 corresponds to updating the learning
                    # rate after every epoch/step.
                "frequency": 1,
                    # If using the `LearningRateMonitor` callback to monitor the
                    # learning rate progress, this keyword can be used to specify
                    # a custom logged name
                    "name": "cycle_lr"
            }
            return [optimizer], [lr_scheduler_config]
        else: # просто учим
            return torch.optim.AdamW(self.parameters(), lr=self.lr)

In [15]:
seq_model = SeqModel(in_ch=len(train_dataset[0][0]), lr = lr)

# Initialize a trainer
trainer = L.Trainer(
    accelerator="auto",
    devices=[1],
    max_epochs=NUM_EPOCHS,
)

# Train the model
trainer.fit(seq_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params | Mode 
---------------------------------------------------------
0 | model       | HumanLegNet      | 1.3 M  | train
1 | criterion   | MSELoss          | 0      | train
2 | val_mse     | MeanSquaredError | 0      | train
3 | val_pearson | PearsonCorrCoef  | 0      | train
---------------------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.293     Total estimated model params size (MB)
119       Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████████████████████████████████████| 1537/1537 [02:01<00:00, 12.60it/s, v_num=29, train_loss=0.651]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|████████| 1537/1537 [02:26<00:00, 10.49it/s, v_num=29, train_loss=0.747, val_acc=0.418, val_pearson=0.562]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|████████| 1537/1537 [02:29<00:00, 10.28it/s, v_num=29, train_loss=0.429, val_acc=0.361, val_pearson=0.634]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|████████| 1537/1537 [02:30<00:00, 10.19it/s, v_num=29, train_loss=0.499, val_acc=0.351, val_pearson=0.649]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|████████| 1537/153

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [17]:
import numpy as np
from scipy import stats

preds = trainer.predict(seq_model, dataloaders=test_loader)
y_real = torch.cat([y for _, y in test_loader])
y_preds = torch.cat(preds).numpy()
stats.pearsonr(y_preds, y_real)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0: 100%|████████████████████████████████████████████████████████| 193/193 [00:13<00:00, 13.87it/s]


PearsonRResult(statistic=np.float64(0.7164604630534733), pvalue=np.float32(0.0))